# Figura rápida en Altair
 Como han contribuido a lo largo del tiempo los 4 paises que más emiten en el 2022 respecto a las emisiones totales e incluir a México

In [ ]:
import pandas as pd
import altair as alt


In [ ]:
# Cargar el conjunto de datos
file_path = '../../data/owid-co2-data.csv'
co2 = pd.read_csv(file_path)

# Filtrar los datos para los países de interés
paises_interes = ['China', 'United States', 'India', 'Russia', 'Mexico', 'World']
co2_filtered = co2[co2['country'].isin(paises_interes)]

# Crear una tabla pivot para los datos
co2_pivot = co2_filtered.pivot(index='year', columns='country', values='co2').fillna(0).reset_index()

# Convertir los datos a un formato largo (long format) para Altair
co2_long = co2_pivot.melt(id_vars='year', var_name='country', value_name='co2')

# Asegurar que el año sea interpretado como temporal
co2_long['year'] = pd.to_datetime(co2_long['year'], format='%Y')
co2_long

In [ ]:
# Crear la gráfica de áreas apiladas
area_chart = alt.Chart(co2_long).transform_filter(
    alt.FieldOneOfPredicate(field='country', oneOf=paises_interes[:-1])
).mark_area().encode(
    x=alt.X('year:T', title='Year', axis=alt.Axis(format='%Y')),
    y=alt.Y('co2:Q', stack='zero', title='CO2 Emissions (million tonnes)'),
    color=alt.Color('country:N', title='Country', scale=alt.Scale(scheme='category10')),
    tooltip=['year:T', 'country:N', 'co2:Q']
).properties(
    width=800,
    height=400
)

# Crear la gráfica de línea para 'World'
line_chart = alt.Chart(co2_long).transform_filter(
    alt.datum.country == 'World'
).mark_line(color='black', size=3).encode(
    x=alt.X('year:T', title='Year', axis=alt.Axis(format='%Y')),
    y=alt.Y('co2:Q', title='CO2 Emissions (million tonnes)'),
    tooltip=['year:T', 'co2:Q']
).properties(
    width=800,
    height=400
)

# Combinar ambas gráficas
chart = alt.layer(area_chart, line_chart).properties(
    title='Emisiones históricas de CO2'
)

# Habilitar la interactividad
chart = chart.interactive()

chart